# Text Mining Project: AUC Courses Recommender
## Desrciption
In this notebook you will find the source code for the Amsterdam University College (AUC) Course Recommender which was made for a Project for the course called Text Mining. (Dalszy opis ...)

## Code
Imports:

In [55]:
import pandas as pd
import nltk
import re
import string
from sklearn.metrics.pairwise import cosine_similarity
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import Word2Vec, KeyedVectors


In [3]:
pd.options.mode.chained_assignment = None

Loading and dealing with the dataset

In [5]:
#Loading the data in as a DataFrame.
data = pd.read_csv("datasets/recommender_dataset.csv")

#Droping the rows which had noting in the row in the course_catalogue_number column.
courses = data.dropna(subset=['course_catalogue_number'])

#Getting out all of the duplicates form the data, just in case there are any.
indices = pd.Series(courses.index, index=courses['course_catalogue_number']).drop_duplicates()

In [11]:
#Removing the stop words from columns: "is_part_of" and "course_description"
for i in range(len(courses.count())):
    courses.loc[i, 'course_name'] = courses.loc[i, 'course_name'].lower()
    courses.loc[i, 'college_graduate'] = courses.loc[i, 'college_graduate'].lower()
    courses.loc[i, 'language_of_instruction'] = courses.loc[i, 'language_of_instruction'].lower()
    courses.loc[i, 'is_part_of'] = courses.loc[i, 'is_part_of'].lower()
    courses.loc[i, 'is_part_of'] = remove_stopwords(courses.loc[i, 'is_part_of'].lower())
    courses.loc[i, 'course_description'] = courses.loc[i, 'course_description'].lower()
    courses.loc[i, 'course_description'] = remove_stopwords(courses.loc[i, 'course_description'].lower())

In [15]:
#Removing punctuation form the text
for i in range(len(courses.count())):
    '''for j in range(len(courses['course_name'][i]))
        if courses['course_name'][i][j] in string.punctuation:'''
    courses.loc[i, 'course_name'] = re.sub(r'[^\w\s]+', '', courses['course_name'][i])
    courses.loc[i, 'is_part_of'] = re.sub(r'[^\w\s]+', '', courses['is_part_of'][i])
    courses.loc[i, 'college_graduate'] = re.sub(r'[^\w\s]+', '', courses['college_graduate'][i])
    courses.loc[i, 'course_description'] = re.sub(r'[^\w\s]+', '', courses['course_description'][i])

In [19]:
#Tokenization of columns: 'course_name', 'is_part_of', 'college_graduate', and 'course_description'
for i in range(len(courses.count())):
    courses.loc[i, 'course_name'] = nltk.tokenize.WordPunctTokenizer().tokenize(courses['course_name'][i])
    courses.loc[i, 'is_part_of'] = nltk.tokenize.WordPunctTokenizer().tokenize(courses['is_part_of'][i])
    courses.loc[i, 'college_graduate'] = nltk.tokenize.WordPunctTokenizer().tokenize(courses['college_graduate'][i])
    courses.loc[i, 'course_description'] = nltk.tokenize.WordPunctTokenizer().tokenize(courses['course_description'][i])

In [47]:
#Making a new column with all of the text from the row.
courses['combined_text'] = courses['course_name'] + courses['is_part_of'] + courses['college_graduate'] + courses['course_description']

In [61]:
#Vectorizing the column courses['combined_text'].
#I pan to use doc2vec to vectorise the tokens for each course

In [29]:
#Getting the cosine similarities between the rows of the matrix (these are besically the similiarities between the courses).
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [63]:
#Test of the cosine similarities to make the first predictions and see if they are somewhat correct.
def recommend_courses(course_id, cosine_sim=cosine_sim, top_n=5):
    idx = indices.get(course_id)
    if idx is None:
        return []
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1] 
    course_indices = [i[0] for i in sim_scores]
    return courses.iloc[course_indices][['course_name', 'course_catalogue_number']]

NameError: name 'cosine_sim' is not defined

In [33]:
recommend_courses('118110146Y')

,course_name,course_catalogue_number
1999,Light on Matter,5113LIOM6Y
2304,Materials for Light-Energy Conversion,5354MFLE6Y
2444,Molecular Spectroscopy,5112MOSS6Y
1890,Introduction to the Energy Transition,900109SCIY
2673,Physics of Sustainable Energy,5092PHSE6Y


In [35]:
recommend_courses('900264SCIY')

,course_name,course_catalogue_number
477,Business Law and Ethics,6012B0428Y
785,Corporate Law,6012B0458Y
1738,International Law of Military Operations,3854INQ8VY
188,Amsterdam Law Clinics,3554ALCLKY
2783,Principles and Foundations of International Law,3554PRFIVY
